In [17]:

spark.sql(
    """
    DROP DATABASE PL7G1_GOLD CASCADE
    """
)

DataFrame[]

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [19]:
# create silver database
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS PL7G1_GOLD LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/'
    """
)

DataFrame[]

In [2]:
# create table in gold
spark.sql(
    """
    DROP TABLE IF EXISTS PL7G1_GOLD.Incendios_Ano
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE PL7G1_GOLD.Incendios_Ano (
        borough string,
        case_year int,
        Appliances long,
        Cooking_Carelessness long,
        Electrical long, 
        Electrical_Devices long,
        Heaters long,
        Hot_Objects long,
        Ignitable long,
        Icendiary long,
        Motors long,
        Natural_Sources long,
        No_Fire long,
        Open_Flame long,
        Other long,
        Sem_Informacao long,
        Smoking long,
        Total_Fires long
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Incendios_Ano'
    """
)


DataFrame[]

In [3]:
from pyspark.sql.functions import substring, avg, sum

# read data from the silver tables
incendios = spark.table("PL7G1_SILVER.Incendios")
incendios.show()

+----------+--------+-----+---------+------------------+--------------------+--------+-----------------------+----------------------+------------------+---------+--------+
|      date|    time|am_pm|battalion|community_District|       neighborhoods|precinct|incident_Classification|cause_Fire_Description|fire_Code_Category|case_Year| borough|
+----------+--------+-----+---------+------------------+--------------------+--------+-----------------------+----------------------+------------------+---------+--------+
|2017-11-09|07:00:00|   AM|       42|                10|Bay Ridge, Dyker ...|      68|   MD Residential St...|     SIU Investigation|             Other|     2017|Brooklyn|
|2017-01-17|08:57:00|   AM|       43|                15|Sheepshead Bay, M...|      61|                  Other|     SIU Investigation|             Other|     2017|Brooklyn|
|2016-11-23|12:12:00|   PM|       42|                10|Bay Ridge, Dyker ...|      68|                    MFA|                   MFA|       

In [4]:
from pyspark.sql.functions import col

incendios = incendios.groupBy("borough","case_year").pivot("fire_code_category").count()
incendios = incendios.na.fill(value=0)
incendios = incendios.withColumn('Total_Fires', incendios.Appliances + col("Cooking Carelessness") + incendios.Electrical+col("Electrical Devices") + incendios.Heaters+ col("Hot Objects")+ incendios.Ignitable + incendios.Incendiary+ incendios.Motors + col("Natural Sources") + col("No Fire")+ col("Open Flame") + incendios.Other + col("Sem Informacao") + incendios.Smoking)

incendios.show()

+--------------+---------+----------+--------------------+----------+------------------+-------+-----------+---------+----------+------+---------------+-------+----------+-----+--------------+-------+-----------+
|       borough|case_year|Appliances|Cooking Carelessness|Electrical|Electrical Devices|Heaters|Hot Objects|Ignitable|Incendiary|Motors|Natural Sources|No Fire|Open Flame|Other|Sem Informacao|Smoking|Total_Fires|
+--------------+---------+----------+--------------------+----------+------------------+-------+-----------+---------+----------+------+---------------+-------+----------+-----+--------------+-------+-----------+
|         Bronx|     2020|        26|                  33|       166|                 5|     10|         27|        7|       349|    13|              3|     57|        42|  121|             0|     86|        945|
|Sem Informacao|     2016|         0|                   0|         0|                 0|      0|          0|        0|         0|     0|            

In [5]:
#Tirar os " " e meter "_"
from pyspark.sql import functions as F
incendios = incendios.select([F.col(col).alias(col.replace(' ', '_')) for col in incendios.columns])
incendios.show()

+--------------+---------+----------+--------------------+----------+------------------+-------+-----------+---------+----------+------+---------------+-------+----------+-----+--------------+-------+-----------+
|       borough|case_year|Appliances|Cooking_Carelessness|Electrical|Electrical_Devices|Heaters|Hot_Objects|Ignitable|Incendiary|Motors|Natural_Sources|No_Fire|Open_Flame|Other|Sem_Informacao|Smoking|Total_Fires|
+--------------+---------+----------+--------------------+----------+------------------+-------+-----------+---------+----------+------+---------------+-------+----------+-----+--------------+-------+-----------+
|         Bronx|     2020|        26|                  33|       166|                 5|     10|         27|        7|       349|    13|              3|     57|        42|  121|             0|     86|        945|
|Sem Informacao|     2016|         0|                   0|         0|                 0|      0|          0|        0|         0|     0|            

In [6]:
# write to delta table
incendios \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema","true") \
    .save("hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Incendios_Ano")

In [7]:
spark.table("PL7G1_GOLD.Incendios_Ano").toPandas()

borough  case_year  Appliances  Cooking_Carelessness  Electrical  \
0            Bronx       2020          26                    33         166   
1   Sem Informacao       2016           0                     0           0   
2            Bronx       2016          37                    62         172   
3         Brooklyn       2018          27                    94         298   
4            Bronx       2018          19                    51         179   
5            Bronx       2019          26                    51         191   
6   Outside of NYC       2019           0                     0           1   
7         Brooklyn       2020          27                    60         280   
8   Sem Informacao       2018           0                     0           0   
9    Staten Island       2017          12                    11          34   
10   Staten Island       2020           6                    16          44   
11  Sem Informacao       2019           0                     0           0   
12  Outside of NYC       2016           0                     0           0   
13          Queens       2019          40                    63         266   
14           Bronx       2021           0                     0           0   
15          Queens       2016          55                    68         287   
16           Bronx       2017          31                    67         141   
17       Manhattan       2021           0                     0           0   
18          Queens       2018          35                    75         210   
19        Brooklyn       2016          62                   123         359   
20          Queens       2017          36                    69         248   
21   Staten Island       2016          17                    17          52   
22       Manhattan       2020          19                    39         130   
23  Sem Informacao       2017           0                     0           0   
24  Outside of NYC       2017           0                     0           0   
25        Brooklyn       2019          40                   106         306   
26   Staten Island       2019          12                    17          44   
27       Manhattan       2017          30                    64         139   
28          Queens       2021           0                     0           0   
29   Staten Island       2018          20                    12          47   
30  Outside of NYC       2018           0                     0           0   
31       Manhattan       2019          32                    71         149   
32        Brooklyn       2017          49                   124         292   
33       Manhattan       2018          39                    50         189   
34       Manhattan       2016          40                    45         189   
35          Queens       2020          25                    63         218   

    Electrical_Devices  Heaters  Hot_Objects  Ignitable  Incendiary  Motors  \
0                    5       10           27          7         349      13   
1                    0        0            0          0           0       0   
2                    6       13           50          5         323      13   
3                    1       25           96          9         443      18   
4                    2       18           52         10         363      14   
5                    1       16           51          9         385      16   
6                    0        0            0          0           0       0   
7                    0       15           58         21         360      18   
8                    0        0            0          0           0       0   
9                    0        6           16          4          73       6   
10                   0        1           13          1          52       4   
11                   0        0            0          0           0       0   
12                   0        0            0          0     

In [8]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Incendios_Ano/`
""").show()

++
||
++
++



In [9]:
spark.sql("""
DROP TABLE IF EXISTS PL7G1_GOLD.Incendios
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS PL7G1_GOLD.Incendios_Ano_Presto (
        borough string,
        case_year int,
        Appliances long,
        Cooking_Carelessness long,
        Electrical long, 
        Electrical_Devices long,
        Heaters long,
        Hot_Objects long,
        Ignitable long,
        Icendiary long,
        Motors long,
        Natural_Sources long,
        No_Fire long,
        Open_Flame long,
        Other long,
        Sem_Informacao long,
        Smoking long,
        Total_Fires long
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Incendios_Ano/_symlink_format_manifest/'
""").show()

++
||
++
++

++
||
++
++

